In [1]:
import os
import yaml
import nbimporter
from datetime import datetime, date
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType,  DateType, TimestampType
from pyspark.sql.functions import to_date, to_timestamp
from lab_table_manager import TableManager
from lab_spark import create_spark_session
from lab_raw_yfinance import RawYFinance
import yfinance as yf
from lab_pg_database_manager import PGDatabaseManager

In [2]:
def parallel_fetch_record(spark, fetch_function, fetch_function_params, record_columm_list):
    try:
        # Distribute (symbol, start_date) pairs across Spark workers
        param_rdd = spark.sparkContext.parallelize(fetch_function_params)    
        
        # Fetch data in parallel
        mapped_record_rdd = param_rdd.flatMap(fetch_function)

        # Convert RDD to DataFrame
        result_df = spark.createDataFrame(mapped_record_rdd, schema=record_columm_list)
        # result_df = spark.createDataFrame(mapped_record_rdd)

        # Show or save the results
        # result_df.show()
        return result_df
    except Exception as e:
        print(f"Error paralleling fetch: {e}")
        return spark.createDataFrame([]) 

In [ ]:
connection_config_file="cfg_connections.yaml"
spark_app_name="raw_yfinance"
spark=create_spark_session(connection_config_file, spark_app_name)
equity_type ="stock"
symbol_date_pairs = [
    ('TSLA', '2024-11-20'),
    ('AAPL', '2024-11-20'),
    ('C', '2024-11-20'),
]
object_schema_config_file="cfg_table_schemas.yaml"
object_type="apis"
object_name="yfinance_stock"
raw_stock_yfiannce = RawYFinance(equity_type, symbol_date_pairs, object_schema_config_file, object_type, object_name)
col_list=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
df = parallel_fetch_record(spark, raw_stock_yfiannce.fetch_yfinance_history_record, symbol_date_pairs, col_list)
df.show()



:: loading settings :: url = jar:file:/opt/workspace/finalytics_spark/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.postgresql#postgresql added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-92214349-5285-47dd-aa97-7d7d8808c3e5;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.0 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.77.1 in central
	found software.amazon.awssdk#bundle;2.29.42 in central
	found software.amazon.awssdk#url-connection-client;2.29.42 in central
	found software.amazon.awssdk#utils;2.

Error paralleling fetch: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob


25/01/03 14:29:38 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [ ]:
df.show()